In [15]:
# Dependencies
import numpy as np
import pandas as pd
import requests
from census import Census
import gmaps
import json
from pprint import pprint
import re

# Census & gmaps API Keys
from config import (api_key, gkey)
c = Census(api_key, year=2018)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [16]:
# Run Census Search to retrieve data on all states
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E",
                          "B19301_001E", "B17001_002E",), {'for': 'state:39'})


#{'for': 'state:39'}) = OHIO

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)


# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate",]]

census_pd.head()

,State,Name,Population,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,39,Ohio,11641879.0,54533.0,30304.0,1645986.0,14.138491


In [17]:
#c.sf1.get('NAME', geo={'for': 'tract:*',
#                       'in': 'state:{} county:170'.format(states.AK.fips)})

census_data_new = c.acs5.get(("NAME", "B19013_001E", "B01003_001E",
                          "B19301_001E", "B17001_002E",),geo={'for':'county:*', 'in':'state:39'})

#census_data_new

In [18]:
censusnew_pd = pd.DataFrame(census_data_new)

# Column Reordering
censusnew_pd = censusnew_pd.rename(columns={"B01003_001E": "Population",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "state": "State", "county" : "County"})
censusnew_pd["Poverty Rate"] = 100 * \
    censusnew_pd["Poverty Count"].astype(
        int) / censusnew_pd["Population"].astype(int)

# Final DataFrame
censusnew_pd = censusnew_pd[["Name", "Population", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate",]]

censusnew_pd["Population"] = censusnew_pd["Population"].map("{:,.0f}".format)
censusnew_pd["Household Income"] = censusnew_pd["Household Income"].map("${:20,.0f}".format)
censusnew_pd["Per Capita Income"] = censusnew_pd["Per Capita Income"].map("${:20,.0f}".format)
censusnew_pd["Poverty Count"] = censusnew_pd["Poverty Count"].map("{:,.0f}".format)
censusnew_pd["Poverty Rate"] = censusnew_pd["Poverty Rate"].map("{:.2f}%".format)

#    censusnew_pd['Name'] = censusnew_pd['Name'].map(lambda x: x.rstrip(', Ohio'))



censusnew_pd.head()

,Name,Population,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,"Gallia County, Ohio","30,195","$ 42,113","$ 22,588","6,608",21.88%
1,"Huron County, Ohio","58,457","$ 51,559","$ 25,375","7,845",13.42%
2,"Athens County, Ohio","65,936","$ 37,778","$ 20,745","17,212",26.10%
3,"Adams County, Ohio","27,878","$ 36,894","$ 21,543","6,230",22.35%
4,"Medina County, Ohio","177,257","$ 74,243","$ 35,979","11,303",6.38%


In [19]:
censusnew_pd['Name'] = censusnew_pd['Name'].map(lambda x: x.rstrip(', Ohio'))
censusnew_pd.head()

,Name,Population,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,Gallia County,"30,195","$ 42,113","$ 22,588","6,608",21.88%
1,Huron County,"58,457","$ 51,559","$ 25,375","7,845",13.42%
2,Athens County,"65,936","$ 37,778","$ 20,745","17,212",26.10%
3,Adams County,"27,878","$ 36,894","$ 21,543","6,230",22.35%
4,Medina County,"177,257","$ 74,243","$ 35,979","11,303",6.38%


In [20]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_pd.to_csv("census_data_states.csv", encoding="utf-8", index=False)
censusnew_pd.to_csv("census_data_counties.csv", encoding="utf-8", index=False)